Author: Michael Van Buren
<br>
Date: 27 September, 2022

### Importing the Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json, ast

sns.set()
%matplotlib inline

In [2]:
df_toyotas = pd.read_csv("hw2_cars.csv", encoding = "unicode-escape")

len(df_toyotas)

859

In [3]:
df_toyotas.dtypes.to_dict()

{'Car Make': dtype('O'),
 'Car Model': dtype('O'),
 'Dealer or Individual': dtype('O'),
 'Price': dtype('float64'),
 'Year': dtype('float64'),
 'Location': dtype('O'),
 'Mileage': dtype('float64'),
 'Doors': dtype('O'),
 'Color': dtype('O'),
 'Transmission': dtype('O'),
 'VIN#': dtype('O'),
 'Engine': dtype('O'),
 'Description': dtype('O'),
 'Web link': dtype('O')}

In [4]:
## make eligible year list - 2008 - 2020
year_list = [2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]

##remove records that do not exist within the eligibility for years to be loaded
df_toyotas_subset = df_toyotas.loc[df_toyotas['Year'].isin(year_list)]

df_toyotas_subset['Year'].value_counts()

2019.0    150
2020.0    105
2018.0     88
2016.0     76
2013.0     65
2017.0     63
2015.0     60
2014.0     57
2012.0     51
2011.0     41
2010.0     31
2009.0     16
2008.0      9
Name: Year, dtype: int64

In [5]:
## create a subset dataframe with fields we are interested in 
df_toyotas_subset = df_toyotas[['Car Model', 'Dealer or Individual', 'Price', 
                                'Year', 'Location', 'Mileage', 'Color', 'Transmission', 'VIN#',
                                'Engine', 'Web link']]

df_toyotas_subset

,Car Model,Dealer or Individual,Price,Year,Location,Mileage,Color,Transmission,VIN#,Engine,Web link
0,Rav4,Dealer,21300.0,2016.0,"Walnut Creek, CA",90441.0,White,Automatic,2T3WFREV0GW274936,2.5 L 4-cylinder,https://sfbay.craigslist.org/eby/ctd/753307708...
1,Rav4,Dealer,10995.0,2009.0,"1125 Detroit Ave, CA",172844.0,White,Automatic,JTMBK31V095081967,"V6, 3.5 Liter",https://sfbay.craigslist.org/eby/ctd/753307624...
2,Highlander,Dealer,21990.0,2012.0,"Lower Bottoms, Oakland, CA",75000.0,White,Automatic,5TDBK3EH9CS140312,"V6, 3.5 Liter",https://sfbay.craigslist.org/eby/ctd/753332603...
3,Corolla,Individual,15590.0,2013.0,Pittsburgh,79767.0,Silver,Automatic,2T1BU4EE6DC074253,1.8-Liter 4-Cylinder DOHC,https://www.carvana.com/vehicle/2417328#inspec...
4,Camry,Dealer,19990.0,2012.0,"San Francisco, CA",76000.0,Blue,Automatic,5TDZA3EH0CS021002,"4-Cyl, 2.7 Liter",https://sfbay.craigslist.org/sfc/ctd/753333249...
...,...,...,...,...,...,...,...,...,...,...,...
854,Rav4,Individual,18863.0,2014.0,NaN,80930.0,Black,Automatic,2T3DFREV9EW220829,NaN,https://www.carfax.com/vehicle/2T3DFREV9EW220829
855,Rav4,Individual,27935.0,2019.0,NaN,NaN,NaN,Automatic,NaN,NaN,https://www.carfax.com/vehicle/2T3F1RFV0KC034955
856,Rav4,Individual,27488.0,2018.0,NaN,NaN,NaN,Automatic,NaN,NaN,https://www.carfax.com/vehicle/JTMDJREV7JD190557
857,Rav4,NaN,16993.0,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.carfax.com/vehicle/2T3DFREV4EW162726


### Cleaning the data

In [6]:
## find number of NaN rows - source: https://www.youtube.com/watch?v=qxpKCBV60U4 @ 8:20/23:40
df_toyotas_subset.isna().sum()

Car Model                1
Dealer or Individual     5
Price                    8
Year                     2
Location                33
Mileage                 12
Color                    7
Transmission             7
VIN#                    27
Engine                  45
Web link                 4
dtype: int64

In [7]:
## expand column width so I can copy the URL and fix data that is missing
## source: https://calmcode.io/til/pandas-max-colwidth.html
pd.set_option('display.max_colwidth', None)

In [8]:
## Investigate NaN 'Car Model' column  - source: https://www.youtube.com/watch?v=qxpKCBV60U4 @ 8:30
df_toyotas_subset[df_toyotas_subset['Car Model'].isna()]

,Car Model,Dealer or Individual,Price,Year,Location,Mileage,Color,Transmission,VIN#,Engine,Web link
365,NaN,Dealer,19949.0,2020.0,"Waukesha, WI",63573.0,Black Sand Pearl Exterior,Automatic,JTDEPRAE4LJ045358,1.8L 4-Cylinder Gas Engine,https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=657096572&allListingType=all-cars&makeCodeList=TOYOTA&city=Pittsburgh&state=PA&zip=15203&location=&searchRadius=0&marketExtension=include&startYear=2008&endYear=2020&isNewSearch=false&showAccelerateBanner=false&sortBy=relevance&numRecords=25&firstRecord=150&dma=&referrer=%2Fcars-for-sale%2Fall-cars%2Ftoyota%2Fpittsburgh-pa-15203%3Fdma%3D%26searchRadius%3D0%26location%3D%26marketExtension%3Dinclude%26startYear%3D2008%26endYear%3D2020%26isNewSearch%3Dfalse%26showAccelerateBanner%3Dfalse%26sortBy%3Drelevance%26numRecords%3D25%26firstRecord%3D150&clickType=spotlight


In [9]:

## rename columnns to add functionality
## source: https://stackoverflow.com/questions/38134643/data-frame-object-has-no-attribute
df_toyotas_subset.rename(columns=
                         {'Car Model' : 'Car_Model'}
                        )


,Car_Model,Dealer or Individual,Price,Year,Location,Mileage,Color,Transmission,VIN#,Engine,Web link
0,Rav4,Dealer,21300.0,2016.0,"Walnut Creek, CA",90441.0,White,Automatic,2T3WFREV0GW274936,2.5 L 4-cylinder,https://sfbay.craigslist.org/eby/ctd/7533077081.html
1,Rav4,Dealer,10995.0,2009.0,"1125 Detroit Ave, CA",172844.0,White,Automatic,JTMBK31V095081967,"V6, 3.5 Liter",https://sfbay.craigslist.org/eby/ctd/7533076248.html
2,Highlander,Dealer,21990.0,2012.0,"Lower Bottoms, Oakland, CA",75000.0,White,Automatic,5TDBK3EH9CS140312,"V6, 3.5 Liter",https://sfbay.craigslist.org/eby/ctd/7533326039.html
3,Corolla,Individual,15590.0,2013.0,Pittsburgh,79767.0,Silver,Automatic,2T1BU4EE6DC074253,1.8-Liter 4-Cylinder DOHC,https://www.carvana.com/vehicle/2417328#inspection-150-point
4,Camry,Dealer,19990.0,2012.0,"San Francisco, CA",76000.0,Blue,Automatic,5TDZA3EH0CS021002,"4-Cyl, 2.7 Liter",https://sfbay.craigslist.org/sfc/ctd/7533332493.html
...,...,...,...,...,...,...,...,...,...,...,...
854,Rav4,Individual,18863.0,2014.0,NaN,80930.0,Black,Automatic,2T3DFREV9EW220829,NaN,https://www.carfax.com/vehicle/2T3DFREV9EW220829
855,Rav4,Individual,27935.0,2019.0,NaN,NaN,NaN,Automatic,NaN,NaN,https://www.carfax.com/vehicle/2T3F1RFV0KC034955
856,Rav4,Individual,27488.0,2018.0,NaN,NaN,NaN,Automatic,NaN,NaN,https://www.carfax.com/vehicle/JTMDJREV7JD190557
857,Rav4,NaN,16993.0,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.carfax.com/vehicle/2T3DFREV4EW162726


In [10]:
## fix the Car Model where was NaN after clicking on the link to view the web link
## source: https://www.geeksforgeeks.org/how-to-replace-values-in-column-based-on-condition-in-pandas/


#df_toyotas_subset.loc[df_toyotas_subset['VIN#'] == 'JTDEPRAE4LJ045358']

#set('Car_Model' == 'Corolla').df_toyotas_subset['VIN#'] == 'JTDEPRAE4LJ045358'

#df_toyotas_subset.loc[df_toyotas_subset['VIN#'] == 'JTDEPRAE4LJ045358'].set(df_toyotas_subset['Car_Model'] == 'Corolla')

##df_toyotas_subset.mask(df_toyotas_subset['Car Model'].isna(), 'Corolla')

##, 'Car Model'] == 'Corolla'

###df_toyotas_subset['Car Model'].replace('NaN', 'Corolla', inplace=True)

In [11]:
df_toyotas_subset[df_toyotas_subset['Car Model'].isna()]

,Car Model,Dealer or Individual,Price,Year,Location,Mileage,Color,Transmission,VIN#,Engine,Web link
365,NaN,Dealer,19949.0,2020.0,"Waukesha, WI",63573.0,Black Sand Pearl Exterior,Automatic,JTDEPRAE4LJ045358,1.8L 4-Cylinder Gas Engine,https://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=657096572&allListingType=all-cars&makeCodeList=TOYOTA&city=Pittsburgh&state=PA&zip=15203&location=&searchRadius=0&marketExtension=include&startYear=2008&endYear=2020&isNewSearch=false&showAccelerateBanner=false&sortBy=relevance&numRecords=25&firstRecord=150&dma=&referrer=%2Fcars-for-sale%2Fall-cars%2Ftoyota%2Fpittsburgh-pa-15203%3Fdma%3D%26searchRadius%3D0%26location%3D%26marketExtension%3Dinclude%26startYear%3D2008%26endYear%3D2020%26isNewSearch%3Dfalse%26showAccelerateBanner%3Dfalse%26sortBy%3Drelevance%26numRecords%3D25%26firstRecord%3D150&clickType=spotlight


In [12]:
## Investigate NaN 'Dealer or Individiaul' column - source: https://www.youtube.com/watch?v=qxpKCBV60U4 @ 8:30
df_toyotas_subset[df_toyotas_subset['Dealer or Individual'].isna()]

,Car Model,Dealer or Individual,Price,Year,Location,Mileage,Color,Transmission,VIN#,Engine,Web link
209,Highlander,NaN,22998.0,2013.0,"CarMax Harrisonburg, VA",95000.0,Black,Automatic,5TDBK3EH9DS180049,"6-cyl, Gas, 3.5L",https://www.carmax.com/car/23234504
624,Camry,NaN,NaN,NaN,"Uniontown, PA",222911.0,Barcelona Red Metallic,Automatic,4T1BF1FK2CU620590,2.5L 4cyl 6A,https://www.edmunds.com/toyota/camry/2012/vin/4T1BF1FK2CU620590/?radius=50
719,Corolla,NaN,21899.0,2022.0,San Jose,0.0,silver,Automatic,5YFEPMAEXNP338036,Regular Unleaded I-4 1.8 L/110,https://www.toyotamilpitas.com/inventory/new-toyota-corolla/
857,Rav4,NaN,16993.0,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.carfax.com/vehicle/2T3DFREV4EW162726
858,Rav4,NaN,NaN,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.carfax.com/vehicle/2T3G1RFV4KW072420


In [13]:
## Investigate NaN 'Price' column - source: https://www.youtube.com/watch?v=qxpKCBV60U4 @ 8:30
df_toyotas_subset[df_toyotas_subset['Price'].isna()]

,Car Model,Dealer or Individual,Price,Year,Location,Mileage,Color,Transmission,VIN#,Engine,Web link
322,Camry,Dealer,NaN,28998.0,"East Haven, CT",71000.0,red,Automatic,4T1B61HK9JU544313,"4-cyl, Gas, 2.5L",https://www.carmax.com/car/23219999
335,Camry,Dealer,NaN,16990.0,"Bridgeville, PA",104473.0,Gray,Automatic,4T1BF1FK1FU050005,"4-Cyl, 2.5L",https://www.carvana.com/vehicle/2470299
419,Highlander,Dealer,NaN,NaN,39998,15000.0,white,Automatic,5TDBZRBH0LS520450,"6-cyl, Gas, 3.5L",https://www.carmax.com/car/23066409
508,Corolla,Dealer,NaN,2016.0,Florida,61262.0,gray,Manual,5YFBURHE7GP369374,1.8L 4-cyl DOHC 16-Valve Dual VVT-i,https://www.carvana.com/vehicle/lt/2336216&store=Carvpit?utm_source=google&utm_medium=vla&utm_campaign=11030956979&utm_content=111143241747&utm_target=pla-1731472881263&utm_creative=461539942938&utm_device=c&utm_adposition=&gclid=Cj0KCQjw94WZBhDtARIsAKxWG-9iNgMmMfDH_W8ioYjq8UbQNGkCSChN1-acjqPpw7boGsjENotmyDcaAsZ1EALw_wcB
527,Highlander,Dealer,NaN,2018.0,"Greensburg, PA",48512.0,Blizzard Pearl,Automatic,NaN,Gas,https://www.edmunds.com/toyota/highlander/2018/vin/5TDJZRFH4JS911457/?radius=50
559,Corolla,Dealer,NaN,2009.0,"1761 West Liberty Avenue Pittsburgh, PA 15226",12000.0,grey,Manual,NaN,4D S,https://www.webuyanycarusa.com/secure/bookappointment/8993ee21-8ef5-4e3c-ba86-135aeeb91020#view
624,Camry,NaN,NaN,NaN,"Uniontown, PA",222911.0,Barcelona Red Metallic,Automatic,4T1BF1FK2CU620590,2.5L 4cyl 6A,https://www.edmunds.com/toyota/camry/2012/vin/4T1BF1FK2CU620590/?radius=50
858,Rav4,NaN,NaN,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,https://www.carfax.com/vehicle/2T3G1RFV4KW072420


In [16]:
## drop NaN values
df_toyotas_subset = df_toyotas_subset['Price'].dropna()

In [17]:
## verify NaN values were dropped
df_toyotas_subset.isna().sum()

0

In [18]:
## Investigate NaN 'Year' column - source: https://www.youtube.com/watch?v=qxpKCBV60U4 @ 8:30
df_toyotas_subset[df_toyotas_subset['Year'].isna()]

KeyError: 'Year'

In [ ]:
## Investigate NaN 'Location' column - source: https://www.youtube.com/watch?v=qxpKCBV60U4 @ 8:30
df_toyotas_subset[df_toyotas_subset['Location'].isna()]

In [ ]:
## Investigate NaN 'Color' column - source: https://www.youtube.com/watch?v=qxpKCBV60U4 @ 8:30
df_toyotas_subset[df_toyotas_subset['Color'].isna()]

In [ ]:
## Investigate NaN 'Transmission' column - source: https://www.youtube.com/watch?v=qxpKCBV60U4 @ 8:30
df_toyotas_subset[df_toyotas_subset['Transmission'].isna()]

In [ ]:
## Investigate NaN 'Vin#' column - source: https://www.youtube.com/watch?v=qxpKCBV60U4 @ 8:30
df_toyotas_subset[df_toyotas_subset['VIN#'].isna()]

In [ ]:
## Investigate NaN 'Engine' column - source: https://www.youtube.com/watch?v=qxpKCBV60U4 @ 8:30
df_toyotas_subset[df_toyotas_subset['Engine'].isna()]

In [ ]:
## Investigate NaN 'Year' column - source: https://www.youtube.com/watch?v=qxpKCBV60U4 @ 8:30
df_toyotas_subset[df_toyotas_subset['Year'].isna()]

1. Creat a df_subset of data removing fields I do not need...
    * (Car Make (All are Toyota))
    * Number of doors (all are 4)
    * Remove Transmission data - most are Automatic / CVT transmissions. Only ~ 1% of cars sold in US are Manual and there are a number of null fields. I don't plan to extract any data based on this column.

1. Make sure all Car Model instances are spelled correctly
1. Make sure the date range is correct - 2008 - 2020
1. Clean duplicate VIN#s - keeping only last instance in my case

In [ ]:
## count how many duplicate Vin#s there are in the dataframe
###df_toyotas_subset[df_toyotas_subset['VIN#'].duplicated(keep=False)]

df_toyotas_subset['VIN#'].duplicated().sum()

In [ ]:
## delete duplicate VIN#s by keeping last
df_toyotas_subset = df_toyotas_subset.drop_duplicates(subset="VIN#", keep="last")

In [ ]:
# verify duplicate Vin# rows were deleted
###df_toyotas_subset[df_toyotas_subset['VIN#'].duplicated(keep=False)]

df_toyotas_subset['VIN#'].duplicated().sum()

### Part 2. EDA

### Part 3. Pose a question that can you believe that can be answered from the data. This should be in a clearly identified section in the same notebook.

Question: Do cars cost more in certain geographic areas?

### Part 4) Suggest some improvements that could be made in order to better collect this data. This should be in a clearly identified section in the same notebook.

Good
<br> 
1. drop down of cars

<br>
<br>

Bad

1. format of mileage should be into specific consistent format
1. requiring fields
1. automatic vs. CVT - transmission should be accurate based on cars that actually exists
1. remove questionst that can be easily answered based on the Car Model - e.g. 4 door for all cars, engine cylinder if you insist on having engine being input - have a drop down based on what is offered by Toyota
1. naming fields should not have space for df.FIELD functionality
1. only allow valid data to be entered - e.g. year of the Toyota looking at, similar to the model